In [ ]:
#ADMIN

# Normalize data using Min-Max Scaler
scaler = MinMaxScaler()
dftf[['normalized_value', 'normalized_PEMAKAIAN_M3']] = scaler.fit_transform(dftf[['normalized_value', 'normalized_PEMAKAIAN_M3']])

# Perform K-Means clustering with optimal number of clusters (k=3)
kmeans = KMeans(n_clusters=3)
kmeans.fit(dftf[['normalized_value', 'normalized_PEMAKAIAN_M3']])
labels = kmeans.labels_

# Add a new column to the dataframe with the cluster labels
dftf['cluster'] = labels

# Calculate distance of each data point to its closest centroid
distances = [np.linalg.norm(x - kmeans.cluster_centers_[labels[i]]) for i, x in enumerate(dftf[['normalized_value', 'normalized_PEMAKAIAN_M3']].values)]

# Identify anomalies based on distance from centroid
anomaly_threshold = np.percentile(distances, 95)  # adjust this value to change the sensitivity of anomaly detection
anomalies = [dist > anomaly_threshold for dist in distances]

# Create new column for anomalies
dftf['anomaly'] = np.where(anomalies, 'anomali', 'normal')

dftf.head()

In [ ]:
file_path = "/content/gdrive/MyDrive/Colab Notebooks/pdam/Hasil_3.csv"
dftf.to_csv(file_path, index=False)

In [ ]:
#SCATTERPLOT

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 3D visualization of clusters
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

colors = ['red', 'green', 'blue']
for i in range(3):
    cluster_points = dftf[dftf['cluster'] == i]
    ax.scatter(cluster_points['normalized_value'], cluster_points['normalized_PEMAKAIAN_M3'], zs=cluster_points.index, c=colors[i], label=f'Cluster {i}')

ax.set_xlabel('Normalized Value')
ax.set_ylabel('Normalized PEMAKAIAN_M3')
ax.set_zlabel('Index')
ax.set_title('K-Means Clustering')
ax.legend()

plt.show()